In [1]:
import numpy as np
import json
import csv
import pandas as pd
import re
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter

In [118]:
year_list = [str(year) for year in range(1978,2019 + 1)]

data = []
with open('../../nominate_movie_meta_data.json', 'r', encoding = 'utf-8') as f:
    json_dict = json.load(f)
    for year in year_list:
        data += json_dict[year]
        
data = pd.DataFrame(data).set_index('id')

def dict2list_other_nominate(series):
    l = []
    other_nominates = series['other_nominates']
    for each in other_nominates:
        if each['prized'] == 1:
            l.append(each['award'])
    return pd.Series([l])

data['other_nominates'] = data.apply(dict2list_other_nominate, axis = 'columns')

data['production_studio'] = data['production_studio'].map(lambda each: re.split(r'[、＝=]',each))

mlb = MultiLabelBinarizer()
onehot = mlb.fit_transform(data['other_nominates'])
columns = mlb.classes_
nomination_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################

performers = []
data['performers'].map(lambda each: performers.extend(each))

count = Counter(performers)
countSeries = pd.Series(count)
selected_performers = countSeries[countSeries > 9].index ######10回以上受賞作品に出演

data['selected_performers'] = data['performers'].map(lambda each: list( set(each) & set(selected_performers) ))

onehot = mlb.fit_transform(data['selected_performers'])
columns = mlb.classes_
selected_performers_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################10回以上出た出演者のonehot


directors = []
data['director'].map(lambda each: directors.extend(each))
    
count = Counter(directors)
countSeries = pd.Series(count)
selected_directors = countSeries[countSeries > 3].index

data['selected_directors'] = data['director'].map(lambda each: list( set(each) & set(selected_directors) ))

onehot = mlb.fit_transform(data['selected_directors'])
columns = mlb.classes_
selected_directors_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################4回以上出た監督のonehot

studio = []
data['production_studio'].map(lambda each: studio.extend(each))

count = Counter(studio)
countSeries = pd.Series(count)
selected_studios = countSeries[countSeries > 20].index ########################### 20回以上受賞

data['selected_studio'] = data['production_studio'].map(lambda each: list( set(each) & set(selected_studios) ))

onehot = mlb.fit_transform(data['selected_studio'])
columns = mlb.classes_
selected_studio_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################4回以上出た監督のonehot

scriptwriters = []
data['scriptwriter'].map(lambda each: scriptwriters.extend(each))

count = Counter(scriptwriters)
countSeries = pd.Series(count)
selected_scriptwriters = countSeries[countSeries > 2].index

data['selected_scriptwriter'] = data['scriptwriter'].map(lambda each: list( set(each) & set(selected_scriptwriters) ))

onehot = mlb.fit_transform(data['selected_scriptwriter'])
columns = mlb.classes_
selected_scriptwriter_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #############

In [139]:
with open('2020.json', 'r', encoding = 'utf-8') as f:
    json_dict = json.load(f)

    
dammy = {
    'id': 0,
    'director': list(selected_directors_onehot.columns),
    'scriptwriter': list(selected_scriptwriter_onehot.columns),
    'performers': list(selected_performers_onehot.columns),
    'screen_time': 893,
    'production_studio': '、'.join(selected_studio_onehot.columns),
    'title': 'dammydammydammy',
    'year': 2999,
    'prize': 1,
    'other_nominates': list(nomination_onehot.columns),
}

target = pd.DataFrame([dammy] + json_dict['2020']).set_index('id')

def dict2list_other_nominate(series):
    l = []
    nullfrag = series.isnull().get('other_nominates')
    if nullfrag is False:
        other_nominates = series.get('other_nominates')
        for each in other_nominates:
            if type(each) is str:
                l.append(each)
            elif type(each) is dict:
                if each['prized'] == 1:
                    l.append(each['award'])
    return pd.Series([l])

target['other_nominates'] = target.apply(dict2list_other_nominate, axis = 1)
target['other_nominates'] = target['other_nominates'].map(lambda each: list( set(each) & set(nomination_onehot.columns) ))


target['production_studio'] = target['production_studio'].map(lambda each: re.split(r'[、＝=]',each))

mlb = MultiLabelBinarizer()
onehot = mlb.fit_transform(target['other_nominates'])
columns = mlb.classes_
target_nomination_onehot = pd.DataFrame(onehot, columns = columns, index = target.index) #####################

#performers = []
#data['performers'].map(lambda each: performers.extend(each))

#count = Counter(performers)
#countSeries = pd.Series(count)
#selected_performers = countSeries[countSeries > 9].index ######10回以上受賞作品に出演

target['selected_performers'] = target['performers'].map(lambda each: list( set(each) & set(selected_performers_onehot.columns) ))

onehot = mlb.fit_transform(target['selected_performers'])
columns = mlb.classes_
target_selected_performers_onehot = pd.DataFrame(onehot, columns = columns, index = target.index) #####################10回以上出た出演者のonehot


#directors = []
#data['director'].map(lambda each: directors.extend(each))
    
#count = Counter(directors)
#countSeries = pd.Series(count)
#selected_directors = countSeries[countSeries > 3].index

target['selected_directors'] = target['director'].map(lambda each: list( set(each) & set(selected_directors_onehot.columns) ))

onehot = mlb.fit_transform(target['selected_directors'])
columns = mlb.classes_
target_selected_directors_onehot = pd.DataFrame(onehot, columns = columns, index = target.index) #####################4回以上出た監督のonehot

#studio = []
#data['production_studio'].map(lambda each: studio.extend(each))

#count = Counter(studio)
#countSeries = pd.Series(count)
#selected_studios = countSeries[countSeries > 20].index ########################### 20回以上受賞

target['selected_studio'] = target['production_studio'].map(lambda each: list( set(each) & set(selected_studio_onehot.columns) ))

onehot = mlb.fit_transform(target['selected_studio'])
columns = mlb.classes_
target_selected_studio_onehot = pd.DataFrame(onehot, columns = columns, index = target.index) #####################4回以上出た監督のonehot

#scriptwriters = []
#data['scriptwriter'].map(lambda each: scriptwriters.extend(each))

#count = Counter(scriptwriters)
#countSeries = pd.Series(count)
#selected_scriptwriters = countSeries[countSeries > 2].index

target['selected_scriptwriter'] = target['scriptwriter'].map(lambda each: list( set(each) & set(selected_scriptwriter_onehot.columns) ))

onehot = mlb.fit_transform(target['selected_scriptwriter'])
columns = mlb.classes_
target_selected_scriptwriter_onehot = pd.DataFrame(onehot, columns = columns, index = target.index) #############

In [160]:
target = target[target['title'] != 'dammydammydammy']
target_nomination_onehot = target_nomination_onehot.loc[target.index]
target_selected_studio_onehot = target_selected_studio_onehot.loc[target.index]
target_selected_directors_onehot = target_selected_directors_onehot.loc[target.index]
target_selected_performers_onehot = target_selected_performers_onehot.loc[target.index]
target_selected_scriptwriter_onehot = target_selected_scriptwriter_onehot.loc[target.index]

In [161]:
target.to_pickle('target.pkl')
target_nomination_onehot.to_pickle('target_nomination_onehot.pkl')
target_selected_studio_onehot.to_pickle('target_selected_studio_onehot.pkl')
target_selected_directors_onehot.to_pickle('target_selected_directors_onehot.pkl')
target_selected_performers_onehot.to_pickle('target_selected_performers_onehot.pkl')
target_selected_scriptwriter_onehot.to_pickle('target_selected_scriptwriter_onehot.pkl')

In [162]:
target = pd.read_pickle('target.pkl')
target_nomination_onehot = pd.read_pickle('target_nomination_onehot.pkl')
target_selected_studio_onehot = pd.read_pickle('target_selected_studio_onehot.pkl')
target_selected_directors_onehot = pd.read_pickle('target_selected_directors_onehot.pkl')
target_selected_performers_onehot = pd.read_pickle('target_selected_performers_onehot.pkl')
target_selected_scriptwriter_onehot = pd.read_pickle('target_selected_scriptwriter_onehot.pkl')